## Load libraries

In [15]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
sys.path.append("../libs")
sys.path.append("../")
from definitions import ROOT_DIR
import utils as utils

# Set pandas options
# Not use scientific notation and use two decimal places and use comma as thousand separator
pd.options.display.float_format = '{:,.2f}'.format


# ETL

## 1. Load dataset

In [16]:
# Load dataset from data_files/internet.xlsx - sheet: 'Ingresos '
df = utils.get_xls_sheet_data('/data_files/internet.xlsx', 'Penetracion-hogares')
df.head().style.format(thousands=',')

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,"2,024",2,Buenos Aires,79.840000
1,"2,024",2,Capital Federal,116.370000
2,"2,024",2,Catamarca,68.810000
3,"2,024",2,Chaco,44.060000
4,"2,024",2,Chubut,86.330000


## 2. Handling missing values

### Look for missing values in all cells

In [17]:
#Find missing values
df.isnull().sum()

Año                             0
Trimestre                       0
Provincia                       0
Accesos por cada 100 hogares    0
dtype: int64

#### There's no missing values

## 3. Look for Duplicates

### Find duplicates for complete rows

In [18]:
#Find duplicates by complete row
df.duplicated().sum()

0

#### There's no complete duplicated rows

### Fin duplicated rows for year and quarter ('Trimestre') and state ('Provincia')

In [19]:
#Find duplicated rows by row, for year and quarter
df.duplicated(subset=['Año', 'Trimestre', 'Provincia']).sum()

0

#### There's no duplicated rows for year and quarter

## 4. Finding outliers

In [20]:
#Finding outliers
df.describe()

,Año,Trimestre,Accesos por cada 100 hogares
count,"1,000.00","1,000.00","1,000.00"
mean,"2,018.80",2.46,52.79
std,3.02,1.11,24.14
min,"2,014.00",1.00,9.51
25%,"2,016.00",1.00,33.96
50%,"2,019.00",2.00,50.13
75%,"2,021.00",3.00,67.01
max,"2,024.00",4.00,124.06


#### In the statistics, we can see there's no outliers values for the column year ("Año"), because the maximum value is 2024 and the data is historical. For the quarter ("Trimestre") column the minimum an maximum values are correct, 1 and 4. For the "Accesos 100/hog" column the distance from max value are close to 75 percentile. No outliers where found.

In [21]:
df.groupby(['Año','Trimestre'])['Provincia'].agg(['count'])

count
Año  Trimestre       
2014 1             16
     2             24
     3             24
     4             24
2015 1             24
     2             24
     3             24
     4             24
2016 1             24
     2             24
     3             24
     4             24
2017 1             24
     2             24
     3             24
     4             24
2018 1             24
     2             24
     3             24
     4             24
2019 1             24
     2             24
     3             24
     4             24
2020 1             24
     2             24
     3             24
     4             24
2021 1             24
     2             24
     3             24
     4             24
2022 1             24
     2             24
     3             24
     4             24
2023 1             24
     2             24
     3             24
     4             24
2024 1             24
     2             24

#### For the year 2014, 1st quarter ("Trimestre") we have 8 provinces without information:

In [22]:
df[(df['Año'] == 2014) & (df['Trimestre'] == 1)]

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
984,2014,1,Buenos Aires,54.57
985,2014,1,Capital Federal,112.25
986,2014,1,Catamarca,24.25
987,2014,1,Chaco,20.72
988,2014,1,Chubut,41.24
989,2014,1,Córdoba,52.52
990,2014,1,Corrientes,22.32
991,2014,1,Entre Ríos,34.37
992,2014,1,Formosa,17.42
993,2014,1,Jujuy,18.42


#### The provinces without information are: ['Salta','San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'] we will input 0 because the age of the lost values makes one think that there were no measurements even in that place.

In [23]:
lost_provinces = ['Salta','San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán']

for lost_province in lost_provinces:
  lost_df = df.head(1).copy()
  lost_df['Año'] = 2014
  lost_df['Trimestre'] = 1
  lost_df['Provincia'] = lost_province
  lost_df['Accesos por cada 100 hogares'] = 0
  df=pd.concat([df, lost_df])

df=df.reset_index()
df[(df['Año'] == 2014) & (df['Trimestre'] == 1)]

,index,Año,Trimestre,Provincia,Accesos por cada 100 hogares
984,984,2014,1,Buenos Aires,54.57
985,985,2014,1,Capital Federal,112.25
986,986,2014,1,Catamarca,24.25
987,987,2014,1,Chaco,20.72
988,988,2014,1,Chubut,41.24
989,989,2014,1,Córdoba,52.52
990,990,2014,1,Corrientes,22.32
991,991,2014,1,Entre Ríos,34.37
992,992,2014,1,Formosa,17.42
993,993,2014,1,Jujuy,18.42


## 5. Data Types

### We will review the data types for each column.

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1008 non-null   int64  
 1   Año                           1008 non-null   int64  
 2   Trimestre                     1008 non-null   int64  
 3   Provincia                     1008 non-null   object 
 4   Accesos por cada 100 hogares  1008 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 39.5+ KB


#### The 'Provincia' column have an object data type, and have strings values. We should change the data type to string.

In [25]:
#Change provincia to string
df['Provincia'] = df['Provincia'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1008 non-null   int64  
 1   Año                           1008 non-null   int64  
 2   Trimestre                     1008 non-null   int64  
 3   Provincia                     1008 non-null   string 
 4   Accesos por cada 100 hogares  1008 non-null   float64
dtypes: float64(1), int64(3), string(1)
memory usage: 39.5 KB


## New columns

### Create a new column with the year and quarter

In [26]:
#Create a new column with the quarter and year
df['Periodo'] = (df['Año'].astype(str) + 'T' + df['Trimestre'].astype(str)).astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1008 non-null   int64  
 1   Año                           1008 non-null   int64  
 2   Trimestre                     1008 non-null   int64  
 3   Provincia                     1008 non-null   string 
 4   Accesos por cada 100 hogares  1008 non-null   float64
 5   Periodo                       1008 non-null   string 
dtypes: float64(1), int64(3), string(2)
memory usage: 47.4 KB


# Save dataset

In [27]:
df=df.reset_index()
df.to_parquet(ROOT_DIR + '/data_files/penetracion_hogares_clean.parquet')